In [7]:
import requests 
import json  
from bs4 import BeautifulSoup
import json
import re
import pyodbc

In [ ]:
   

# --- 1. Configuration (les paramètres de ton pipeline) ---
SUBREDDIT = "headphones"
MOT_CLE = "Sony XM5" # Un mot-clé de ton fichier config

# --- 2. Construction de l'URL et des Paramètres ---
# On utilise l'endpoint de RECHERCHE de Reddit, au format JSON
url = f"https://www.reddit.com/r/{SUBREDDIT}/search.json"

# Paramètres de la recherche :
params = {
    'q': MOT_CLE,        # 'q' = query (le mot-clé que tu cherches)
    'sort': 'new',       # 'new' = trier par "plus récent" (parfait pour ton pipeline)
    'restrict_sr': 'true', # 'true' = restreindre la recherche à ce subreddit
    'limit': 10          # On veut 10 résultats
}

# !! TRÈS IMPORTANT !!
# Reddit bloque les scripts qui n'ont pas de "User-Agent".
# On doit simuler un navigateur pour être poli et éviter un blocage.
headers = {
    'User-Agent': 'MonProjetDataEngineering-v0.1'
}

# --- 3. Exécution de l'Appel API ---
print(f"Appel de l'API Reddit pour '{MOT_CLE}' dans r/{SUBREDDIT}...")

try:
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status() # Vérifie s'il y a eu une erreur (ex: 404, 500)

    # --- 4. Affichage du Résultat ---
    data = response.json() # Convertit la réponse texte en objet JSON

    # Les posts sont imbriqués dans cette structure
    posts = data['data']['children']

    if not posts:
        print(f"\n Pas de nouveaux posts trouvés pour '{MOT_CLE}'. ---")
    else:
        print(f"\n{len(posts)} posts trouvés : ---")
        
        # --- 5. Début de la Transformation (ce que fera ton ETL) ---
        for i, post in enumerate(posts):
            post_data = post['data']
            
            titre = post_data['title']
            texte_brut = post_data['selftext'] # Le corps du post
            
            print(f"\n--- Post {i+1} (ID: {post_data['id']}) ---")
            print(f"Titre: {titre}")
            
            
            # Ton script d'analyse de sentiment lira ce titre et ce texte
            # ... (Étape suivante: appliquer VADER ou TextBlob ici) ...


except requests.exceptions.HTTPError as err:
    print(f"\n--- ❌ ERREUR HTTP : {err} ---")
except requests.exceptions.RequestException as e:
    print(f"\n--- ❌ ERREUR de Connexion : {e} ---")
except KeyError:
    print("\n--- ❌ ERREUR de Parsing JSON ---")
    print("La structure de la réponse de Reddit a peut-être changé.")
    print("Réponse brute reçue :", response.text[:200] + "...")

Appel de l'API Reddit pour 'Sony XM5' dans r/headphones...

--- ✅ SUCCÈS ! 10 posts trouvés : ---

--- Post 1 (ID: 1odad1t) ---
Titre: What are these?

--- Post 2 (ID: 1occfmu) ---
Titre: Audiophile Verdict of Bose QC Ultra 2 Headphones: DO NOT BUY 1ST GEN!

--- Post 3 (ID: 1o1oqjf) ---
Titre: Motion sickness from Sennheiser M4, should i switch to other brands?

--- Post 4 (ID: 1nuylfe) ---
Titre: Just bought Arya Stealth coming from Sony XM5 over ears. Will I notice a difference?

--- Post 5 (ID: 1nf3m4a) ---
Titre: Why do I struggle to enjoy over-ear headphones like the Sony XM4 or XM5, and how can I get used to them and start liking the experience?

--- Post 6 (ID: 1n16nhe) ---
Titre: How to disinfect/clean ear cups?

--- Post 7 (ID: 1n13i3p) ---
Titre: Bose QCU #FAIL Sony #FAIL are there any decent options left?

--- Post 8 (ID: 1myfywb) ---
Titre: Sonos Ace!

--- Post 9 (ID: 1mjxnrh) ---
Titre: Sony xm5 pros and cons

--- Post 10 (ID: 1m27bsa) ---
Titre: XM5


In [ ]:

# --- 1. Configuration ---
URL_DECOUVERTE = "https://www.fnac.com/Casque-Bluetooth-sans-fil/Casque-par-usage/nsh450503/w-4?SDM=list&ssi=6&sso=2"

# --- NOUVEAU : En-têtes plus complets ---
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7', # Préférer le français
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.google.com/', # Simule une venue depuis Google
    'DNT': '1', # Do Not Track
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}

# --- 2. Scraping de la page ---
print(f"Appel de l'URL : {URL_DECOUVERTE}...")
try:
    # Utilisation d'une session pour potentiellement gérer les cookies si nécessaire
    session = requests.Session()
    session.headers.update(headers)
    response = session.get(URL_DECOUVERTE)
    response.raise_for_status() # Lève une exception pour les codes 4xx/5xx

    # ... (le reste de ton code pour parser avec BeautifulSoup reste le même) ...
    # ... (trouver le div#FnacContent, extraire data-state, parser le JSON...) ...
    
    soup = BeautifulSoup(response.content, 'html.parser')
    main_div = soup.find('div', id='FnacContent')
    
    if not main_div or 'data-state' not in main_div.attrs:
        print("\n--- ❌ ERREUR : Impossible de trouver le JSON 'data-state'. ---")
    else:
        json_string = main_div['data-state']
        data = json.loads(json_string)
        references = data.get('references', [])
        
        if not references:
            print("\n--- ⚠️ Pas de produits trouvés dans le JSON 'references'. ---")
        else:
            print(f"\n--- ✅ SUCCÈS : {len(references)} produits découverts via JSON ---")
            liste_produits = []
            for ref in references:
                prid = ref.get('prid')
                if prid:
                    # !! VÉRIFIE CE FORMAT D'URL !!
                    product_url = f"https://www.fnac.com/a{prid}/w-4" 
                    liste_produits.append({"prid": prid, "url": product_url})

            print("\n--- Liste des produits découverts (PRID et URL) : ---")
            for p in liste_produits[:5]: 
                 print(p)


except requests.exceptions.HTTPError as err:
    # Affichage plus détaillé de l'erreur 403
    print(f"\n--- ❌ ERREUR HTTP : {err} ---") 
    if response.status_code == 403:
        print("   Cause probable : Blocage anti-scraping par le serveur.")
        print("   Vérifie les 'headers'. Si ça persiste, une solution de proxy pourrait être nécessaire.")
    else:
        print("   Le serveur a retourné une erreur.")
except json.JSONDecodeError:
    print("\n--- ❌ ERREUR : Impossible de parser le JSON dans 'data-state'. ---")
except Exception as e:
    print(f"\n--- ❌ ERREUR INATTENDUE : {e} ---")

Appel de l'URL : https://www.fnac.com/Casque-Bluetooth-sans-fil/Casque-par-usage/nsh450503/w-4?SDM=list&ssi=6&sso=2...

--- ❌ ERREUR HTTP : 403 Client Error: Forbidden for url: https://www.fnac.com/Casque-Bluetooth-sans-fil/Casque-par-usage/nsh450503/w-4?SDM=list&ssi=6&sso=2 ---
   Cause probable : Blocage anti-scraping par le serveur.
   Vérifie les 'headers'. Si ça persiste, une solution de proxy pourrait être nécessaire.


In [ ]:


# --- 1. Configuration ---
URL_TEST = "https://www.vandenborre.be/fr/mp3-casque-ecouteurs/casque"

# En-têtes pour simuler un navigateur
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.google.com/',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}

# --- 2. Exécution du Test ---
print(f"Tentative d'accès à : {URL_TEST}...")
try:
    session = requests.Session()
    session.headers.update(headers)
    response = session.get(URL_TEST, timeout=10) # Ajout d'un timeout
    response.raise_for_status() # Lève une exception pour les codes 4xx/5xx

    # --- 3. Résultat ---
    print(f"\n--- ✅ SUCCÈS ! Code statut : {response.status_code} ---")
    print("Le site semble accessible au scraping basique.")
    # On pourrait ajouter ici une vérification rapide du contenu pour être sûr
    # print(f"Contenu reçu (premiers 200 chars): {response.text[:200]}...")

except requests.exceptions.HTTPError as err:
    print(f"\n--- ❌ ERREUR HTTP : {err} ---")
    if response.status_code == 403:
        print("   Cause probable : Blocage anti-scraping (similaire à FNAC).")
    else:
        print(f"   Le serveur a retourné une erreur {response.status_code}.")
except requests.exceptions.Timeout:
     print("\n--- ❌ ERREUR : La requête a expiré (Timeout). Le serveur est peut-être lent ou bloque.")
except requests.exceptions.RequestException as e:
    print(f"\n--- ❌ ERREUR de Connexion : {e} ---")
except Exception as e:
    print(f"\n--- ❌ ERREUR INATTENDUE : {e} ---")

Tentative d'accès à : https://www.vandenborre.be/fr/mp3-casque-ecouteurs/casque...

--- ✅ SUCCÈS ! Code statut : 200 ---
Le site semble accessible au scraping basique.


In [ ]:

URL_DECOUVERTE = "https://www.vandenborre.be/fr/mp3-casque-ecouteurs/casque"
BASE_URL = "https://www.vandenborre.be"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.google.com/',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}

print(f"Scraping : {URL_DECOUVERTE}...")
try:
    session = requests.Session()
    session.headers.update(headers)
    response = session.get(URL_DECOUVERTE, timeout=15)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    product_containers = soup.find_all('div', {'class': 'js-product-container'})

    if not product_containers:
        print("\n--- ⚠️ Aucun conteneur produit trouvé avec 'div.js-product-container'. Vérifie les sélecteurs. ---")

    print(f"\n--- ✅ {len(product_containers)} conteneurs produits trouvés ---")

    produits_decouverts = []

    for container in product_containers:
        product_id = container.get('data-productid')
        if not product_id:
            continue

        product_data = {"product_id": product_id}

        # URL et Nom
        name_tag = container.find('h2', {'class': 'productname'})
        link_tag = container.find('a', {'class': 'js-product-click'})
        if name_tag and link_tag and link_tag.get('href'):
            product_data["name"] = name_tag.text.strip()
            # Construit l'URL complète
            relative_url = link_tag['href']
            if relative_url.startswith('//'):
                product_data["url"] = f"https:{relative_url}"
            elif relative_url.startswith('/'):
                 product_data["url"] = f"{BASE_URL}{relative_url}"
            else:
                 product_data["url"] = relative_url # Au cas où elle serait déjà complète
        else:
            product_data["name"] = "Nom non trouvé"
            product_data["url"] = "URL non trouvée"

        # Prix
        price_tag = container.find('span', {'class': 'current'})
        if price_tag:
            price_text = price_tag.text.strip().replace('€', '').replace(',', '.').replace('\xa0', '').replace(' ', '')
            try:
                product_data["price"] = float(re.sub(r'[^\d\.]', '', price_text))
            except (ValueError, TypeError):
                 product_data["price"] = None
        else:
            product_data["price"] = None

        # Note et Avis
        rating_score_tag = container.find('div', {'class': 'rating-score'})
        review_count_tag = container.find('div', {'class': 'rating-reviews-amount'})

        if rating_score_tag and rating_score_tag.find('strong'):
            rating_text = rating_score_tag.find('strong').text.strip().replace(',', '.')
            try:
                product_data["rating"] = float(rating_text)
            except (ValueError, TypeError):
                 product_data["rating"] = None
        else:
             product_data["rating"] = None

        if review_count_tag and review_count_tag.find('a'):
            review_text = review_count_tag.find('a').text.strip()
            count_match = re.search(r'\((\d+)\)', review_text)
            if count_match:
                 try:
                    product_data["review_count"] = int(count_match.group(1))
                 except (ValueError, TypeError):
                    product_data["review_count"] = None
            else:
                 product_data["review_count"] = None
        else:
            product_data["review_count"] = None
            
        # Marque (simple extraction du premier mot du nom)
        if product_data["name"] != "Nom non trouvé":
             product_data["brand"] = product_data["name"].split(' ')[0]
        else:
             product_data["brand"] = None


        produits_decouverts.append(product_data)

    print("\n--- Données extraites (5 premiers produits) : ---")
    if produits_decouverts:
        print(json.dumps(produits_decouverts[:5], indent=2, ensure_ascii=False))
    else:
        print("Aucun produit n'a pu être extrait.")


except requests.exceptions.HTTPError as err:
    print(f"\n--- ❌ ERREUR HTTP : {err} ---")
    if response and response.status_code == 403:
        print("   Cause : Blocage anti-scraping.")
except Exception as e:
    print(f"\n--- ❌ ERREUR INATTENDUE : {e} ---")

Scraping : https://www.vandenborre.be/fr/mp3-casque-ecouteurs/casque...

--- ✅ 29 conteneurs produits trouvés ---

--- Données extraites (5 premiers produits) : ---
[
  {
    "product_id": "7819145",
    "name": "JBL TUNE 770NC BLACK",
    "url": "https://www.vandenborre.be/fr/casque/jbl-tune-770nc-black",
    "price": 89.0,
    "rating": 4.4,
    "review_count": 70,
    "brand": "JBL"
  },
  {
    "product_id": "7683081",
    "name": "SONY WH-1000XM6 NOIR",
    "url": "https://www.vandenborre.be/fr/casque/sony-wh-1000xm6-noir",
    "price": 449.0,
    "rating": 4.8,
    "review_count": 17,
    "brand": "SONY"
  },
  {
    "product_id": "7762429",
    "name": "JBL LIVE 770NC NOIR",
    "url": "https://www.vandenborre.be/fr/casque/jbl-live-770nc-noir",
    "price": 120.0,
    "rating": 4.5,
    "review_count": 147,
    "brand": "JBL"
  },
  {
    "product_id": "7825773",
    "name": "JBL TUNE 520 BT BLACK",
    "url": "https://www.vandenborre.be/fr/casque/jbl-tune-520-bt-black",
    "pr

In [ ]:


# --- 1. Configuration ---
SUBREDDIT = "headphones"
# Liste des mots-clés correspondant aux produits trouvés
PRODUITS_A_TESTER = [
    "JBL Tune 770NC", 
    "Sony WH-1000XM6", 
    "JBL Live 770NC" 
]

headers = {
    'User-Agent': 'MonProjetDataEngineering-TestMention-v0.1' 
}

# --- 2. Boucle de Test ---
print(f"Test de mentions dans r/{SUBREDDIT}...\n")
produits_avec_mentions = 0

for mot_cle in PRODUITS_A_TESTER:
    print(f"--- Recherche de '{mot_cle}' ---")
    
    url = f"https://www.reddit.com/r/{SUBREDDIT}/search.json"
    params = {
        'q': mot_cle,
        'sort': 'new',
        'restrict_sr': 'true',
        'limit': 5 # On ne cherche que 5 posts pour ce test
    }

    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()
        posts = data['data']['children']

        if not posts:
            print(f"   -> ⚠️ Aucune mention récente trouvée pour '{mot_cle}'.")
        else:
            print(f"   -> ✅ Trouvé {len(posts)} mentions récentes pour '{mot_cle}'.")
            # Afficher le titre du premier post trouvé pour vérification
            print(f"      Exemple: '{posts[0]['data']['title'][:80]}...'")
            produits_avec_mentions += 1
            
    except requests.exceptions.HTTPError as err:
        print(f"   -> ❌ ERREUR HTTP lors de la recherche de '{mot_cle}': {err}")
    except Exception as e:
        print(f"   -> ❌ ERREUR INATTENDUE lors de la recherche de '{mot_cle}': {e}")
        
    # Pause de politesse pour respecter les limites de Reddit
    time.sleep(1) 

# --- 3. Conclusion du Test ---
print("\n--- Résultat du Test ---")
if produits_avec_mentions > 0:
    print(f"✅ Confirmation : Au moins {produits_avec_mentions}/{len(PRODUITS_A_TESTER)} produits testés ont des mentions récentes sur Reddit.")
    print("   -> La Source 3 (Sentiment) semble viable.")
else:
     print("❌ Problème : Aucun des produits testés n'a de mention récente sur Reddit.")
     print("   -> La Source 3 (Sentiment Reddit) pourrait être difficile à alimenter pour ce marché.")
     print("   -> Envisage d'élargir les mots-clés ou de changer de subreddit.")

Test de mentions dans r/headphones...

--- Recherche de 'JBL Tune 770NC' ---
   -> ✅ Trouvé 5 mentions récentes pour 'JBL Tune 770NC'.
      Exemple: 'I have no idea which headphone to go ahead with after this one broke...'
--- Recherche de 'Sony WH-1000XM6' ---
   -> ✅ Trouvé 5 mentions récentes pour 'Sony WH-1000XM6'.
      Exemple: 'first impression on sony’s WH-1000XM6, i have one complaint...'
--- Recherche de 'JBL Live 770NC' ---
   -> ✅ Trouvé 5 mentions récentes pour 'JBL Live 770NC'.
      Exemple: 'I saw someone wearing the xm4’s...'

--- Résultat du Test ---
✅ Confirmation : Au moins 3/3 produits testés ont des mentions récentes sur Reddit.
   -> La Source 3 (Sentiment) semble viable.


In [10]:
import requests
from bs4 import BeautifulSoup
import json
import re

URL_PRODUIT = "https://www.vandenborre.be/fr/casque/jbl-tune-770nc-black"
BASE_URL = "https://www.vandenborre.be"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
}

print(f"Scraping (JSON Caché) : {URL_PRODUIT}...")
try:
    session = requests.Session()
    session.headers.update(headers)
    response = session.get(URL_PRODUIT, timeout=15)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    # --- 3. Extraction du JSON (Version corrigée) ---
    
    # Trouve TOUS les scripts JSON-LD
    json_scripts = soup.find_all('script', {'type': 'application/ld+json'})
    
    product_data = None # Variable pour stocker le bon JSON

    for script in json_scripts:
        if not script.string:
            continue
            
        try:
            data = json.loads(script.string)
            
            # Cas 1: Le JSON est un dictionnaire
            if isinstance(data, dict) and data.get("@type") == "Product":
                product_data = data
                break # On a trouvé le bon JSON, on arrête la boucle
                
            # Cas 2: Le JSON est une liste de dictionnaires
            if isinstance(data, list):
                for item in data:
                    if isinstance(item, dict) and item.get("@type") == "Product":
                        product_data = item
                        break # On a trouvé le bon JSON
            if product_data:
                break

        except json.JSONDecodeError:
            continue # Ignorer les scripts JSON mal formés

    # --- 4. Affichage du Résultat ---
    if not product_data:
        print("\n--- ❌ ERREUR : Impossible de trouver le JSON '@type': 'Product' dans la page. ---")
    else:
        print("\n--- ✅ SUCCÈS ! Données JSON 'Product' extraites : ---")
        
        # Données pour Dim_Product
        print("\n--- Pour Dim_Product (Catalogue) ---")
        print(f"SKU: {product_data.get('sku')}")
        print(f"Nom: {product_data.get('name')}")
        print(f"Marque: {product_data.get('brand', {}).get('name')}")
        print(f"Catégorie: {product_data.get('category')}")
        
        # Données pour Fact_Marketplace_Snapshot
        print("\n--- Pour Fact_Marketplace_Snapshot (Performances) ---")
        print(f"Prix: {product_data.get('offers', {}).get('price')}")
        print(f"Note: {product_data.get('aggregateRating', {}).get('ratingValue')}")
        print(f"Nb Avis: {product_data.get('aggregateRating', {}).get('reviewCount')}")
        print(f"Dispo: {product_data.get('offers', {}).get('availability')}")


except requests.exceptions.HTTPError as err:
    print(f"\n--- ❌ ERREUR HTTP : {err} ---")
except Exception as e:
    print(f"\n--- ❌ ERREUR INATTENDUE : {e} ---")

Scraping (JSON Caché) : https://www.vandenborre.be/fr/casque/jbl-tune-770nc-black...

--- ✅ SUCCÈS ! Données JSON 'Product' extraites : ---

--- Pour Dim_Product (Catalogue) ---
SKU: 7819145
Nom: JBL TUNE 770NC BLACK
Marque: JBL
Catégorie: Casque audio

--- Pour Fact_Marketplace_Snapshot (Performances) ---
Prix: 89
Note: 4.4
Nb Avis: 70
Dispo: https://schema.org/InStock


In [ ]:

DB_CONFIG = {
    'server': 'LAPTOP-VT8FTHG2\DATAENGINEER', # ex: '.\SQLEXPRESS' ou 'MON-PC\NOM_INSTANCE'
    'database': 'Projet_Market_Staging',
    'driver': '{ODBC Driver 17 for SQL Server}',
    'connection_string': (
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=LAPTOP-VT8FTHG2\DATAENGINEER;" # Doit être le même que 'server'
        "DATABASE=Projet_Market_Staging;"
        "Trusted_Connection=yes;" # La ligne clé pour l'authentification Windows
    )
}

# --- 2. Configuration du Scraper ---
URL_DECOUVERTE = "https://www.vandenborre.be/fr/mp3-casque-ecouteurs/casque" 
BASE_URL = "https://www.vandenborre.be"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
}


try:
    # Etape E (Extraction)
    print(f"Scraping : {URL_DECOUVERTE}...")
    session = requests.Session()
    session.headers.update(headers)
    response = session.get(URL_DECOUVERTE, timeout=15)
    response.raise_for_status()
    print("   -> ✅ Page 'Découverte' scrapée avec succès.")

    soup = BeautifulSoup(response.content, 'html.parser')

    # Etape T (Transformation)
    product_containers = soup.find_all('div', {'class': 'js-product-container'})
    
    if not product_containers:
        print("--- Fin : Aucun conteneur produit trouvé. ---")
        exit()
        
    print(f"   -> ✅ {len(product_containers)} produits trouvés sur la page.")

    # Etape L (Load)
    print("Connexion à la base de données Staging (Auth Windows)...")
    
    # Correction de la chaîne de connexion pour l'adapter à tes infos
    conn_str = DB_CONFIG['connection_string'].replace('NOM_DE_TON_SERVEUR_SQL', DB_CONFIG['server'])
                                                
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    print("   -> ✅ Connecté à SQL Server.")

    insert_count = 0
    for container in product_containers:
        sku = container.get('data-productid')
        if not sku:
            continue

        link_tag = container.find('a', {'class': 'js-product-click'})
        if not link_tag or not link_tag.get('href'):
            continue
            
        # Reconstruction de l'URL
        relative_url = link_tag['href']
        if relative_url.startswith('//'):
            product_url = f"https:{relative_url}"
        else:
            product_url = f"{BASE_URL}{relative_url}"

        # On vérifie si ce produit est déjà en attente de scraping
        cursor.execute("SELECT 1 FROM Staging_Scraping_Queue WHERE ProductID_SKU = ? AND Status = 'pending'", (sku))
        if cursor.fetchone() is None:
            # Nouveau produit à scraper : on l'ajoute à la file d'attente
            cursor.execute(
                "INSERT INTO Staging_Scraping_Queue (ProductID_SKU, ProductURL, Status, DiscoveredAt) VALUES (?, ?, 'pending', GETDATE())",
                (sku, product_url)
            )
            insert_count += 1
    
    conn.commit()
    print(f"   -> ✅ {insert_count} nouveaux produits insérés dans Staging_Scraping_Queue.")
    
except requests.exceptions.HTTPError as err:
    print(f"\n--- ❌ ERREUR HTTP : {err} ---")
except pyodbc.Error as ex:
    sqlstate = ex.args[0]
    print(f"\n--- ❌ ERREUR SQL Server : {sqlstate} ---")
    print("Vérifie tes 'DB_CONFIG': nom du serveur, nom de la base, et que le driver ODBC est installé.")
    print(f"Chaîne de connexion tentée : {conn_str}")
except Exception as e:
    print(f"\n--- ❌ ERREUR INATTENDUE : {e} ---")
finally:
    if 'cursor' in locals() and cursor:
        cursor.close()
    if 'conn' in locals() and conn:
        conn.close()
        print("Connexion SQL Server fermée.")